# Lasso Regression Demo
This notebook shows how L1 regularization encourages sparsity (feature selection) using scikit-learn's built-in diabetes dataset.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score

sns.set(style='whitegrid')
dataset = load_diabetes(as_frame=True)
X, y = dataset.data, dataset.target
X.shape, y.shape

## Why scale before Lasso?
L1 penalizes the absolute size of coefficients. If features are on different scales, the penalty would unfairly shrink some features more than others. `StandardScaler` ensures each feature has unit variance.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("lasso", Lasso(alpha=0.1, max_iter=5000)),
])

pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_test)

mse = mean_squared_error(y_test, preds)
r2 = r2_score(y_test, preds)
mse, r2

In [ ]:
coef = pipeline.named_steps['lasso'].coef_
features = dataset.feature_names
nonzero = np.count_nonzero(coef)
print(f'Non-zero coefficients: {nonzero}/{len(coef)}')

plt.figure(figsize=(8,4))
sns.barplot(x=features, y=coef, palette='Blues_d')
plt.axhline(0, color='black', linewidth=1)
plt.xticks(rotation=45)
plt.title('Lasso Coefficients (alpha=0.1)')
plt.tight_layout()
plt.show()

## Effect of alpha
Try a few alpha values to see sparsity change. Larger alphas push more coefficients to zero.

In [ ]:
def run_alpha(alpha):
    pipe = Pipeline([('scaler', StandardScaler()), ('lasso', Lasso(alpha=alpha, max_iter=5000))])
    pipe.fit(X_train, y_train)
    coef = pipe.named_steps['lasso'].coef_
    nonzero = np.count_nonzero(coef)
    preds = pipe.predict(X_test)
    r2 = r2_score(y_test, preds)
    return nonzero, r2

alphas = [0.01, 0.1, 0.5, 1.0]
results = [(a, *run_alpha(a)) for a in alphas]
results